# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 2:59PM,240242,20,SSNA-8179078,Sartorius Stedim North America,Released
1,Jun 22 2022 2:59PM,240241,16,SEQIRUS-8179072,Seqirus,Released
2,Jun 22 2022 2:57PM,240240,10,MSP211550,"Methapharm, Inc.",Released
3,Jun 22 2022 2:57PM,240240,10,MSP211561,"Methapharm, Inc.",Released
4,Jun 22 2022 2:51PM,240239,20,6619912,Sartorius Stedim North America,Released
5,Jun 22 2022 2:49PM,240238,12,HH6281,Hush Hush,Released
6,Jun 22 2022 2:46PM,240237,16,SEQIRUS-8179069,Seqirus,Released
7,Jun 22 2022 2:43PM,240236,10,8178389,Eywa Pharma Inc.,Released
8,Jun 22 2022 2:43PM,240236,10,8178393,Eywa Pharma Inc.,Released
9,Jun 22 2022 2:43PM,240236,10,8178398,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
53,240238,Released,1
54,240239,Released,1
55,240240,Released,2
56,240241,Released,1
57,240242,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240238,NaN,NaN,1.0
240239,NaN,NaN,1.0
240240,NaN,NaN,2.0
240241,NaN,NaN,1.0
240242,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0.0,1.0,0.0
240133,10.0,14.0,14.0
240134,0.0,0.0,1.0
240135,0.0,0.0,1.0
240137,7.0,10.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0,1,0
240133,10,14,14
240134,0,0,1
240135,0,0,1
240137,7,10,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,0,1,0
1,240133,10,14,14
2,240134,0,0,1
3,240135,0,0,1
4,240137,7,10,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,,1,
1,240133,10,14,14
2,240134,,,1
3,240135,,,1
4,240137,7,10,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America
1,Jun 22 2022 2:59PM,240241,16,Seqirus
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc."
4,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America
5,Jun 22 2022 2:49PM,240238,12,Hush Hush
6,Jun 22 2022 2:46PM,240237,16,Seqirus
7,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.
24,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America
25,Jun 22 2022 2:30PM,240233,10,Emerginnova
26,Jun 22 2022 2:22PM,240232,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America,,,1
1,Jun 22 2022 2:59PM,240241,16,Seqirus,,,1
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc.",,,2
3,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,,,1
4,Jun 22 2022 2:49PM,240238,12,Hush Hush,,,1
5,Jun 22 2022 2:46PM,240237,16,Seqirus,,,1
6,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.,,,17
7,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,,,1
8,Jun 22 2022 2:30PM,240233,10,Emerginnova,,,1
9,Jun 22 2022 2:22PM,240232,19,"AdvaGen Pharma, Ltd",,4,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America,1,,
1,Jun 22 2022 2:59PM,240241,16,Seqirus,1,,
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc.",2,,
3,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1,,
4,Jun 22 2022 2:49PM,240238,12,Hush Hush,1,,
5,Jun 22 2022 2:46PM,240237,16,Seqirus,1,,
6,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.,17,,
7,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,1,,
8,Jun 22 2022 2:30PM,240233,10,Emerginnova,1,,
9,Jun 22 2022 2:22PM,240232,19,"AdvaGen Pharma, Ltd",7,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America,1,,
1,Jun 22 2022 2:59PM,240241,16,Seqirus,1,,
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc.",2,,
3,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1,,
4,Jun 22 2022 2:49PM,240238,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 22 2022 2:59PM,240241,16,Seqirus,1.0,NaN,NaN
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc.",2.0,NaN,NaN
3,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1.0,NaN,NaN
4,Jun 22 2022 2:49PM,240238,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:59PM,240242,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 22 2022 2:59PM,240241,16,Seqirus,1.0,0.0,0.0
2,Jun 22 2022 2:57PM,240240,10,"Methapharm, Inc.",2.0,0.0,0.0
3,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1.0,0.0,0.0
4,Jun 22 2022 2:49PM,240238,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3122551,60.0,5.0,5.0
12,480445,4.0,0.0,0.0
15,1681195,37.0,36.0,29.0
16,960875,3.0,1.0,0.0
18,480450,1.0,1.0,0.0
19,1681286,28.0,9.0,0.0
20,1441343,32.0,15.0,0.0
21,480408,2.0,0.0,0.0
22,240218,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3122551,60.0,5.0,5.0
1,12,480445,4.0,0.0,0.0
2,15,1681195,37.0,36.0,29.0
3,16,960875,3.0,1.0,0.0
4,18,480450,1.0,1.0,0.0
5,19,1681286,28.0,9.0,0.0
6,20,1441343,32.0,15.0,0.0
7,21,480408,2.0,0.0,0.0
8,22,240218,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,5.0,5.0
1,12,4.0,0.0,0.0
2,15,37.0,36.0,29.0
3,16,3.0,1.0,0.0
4,18,1.0,1.0,0.0
5,19,28.0,9.0,0.0
6,20,32.0,15.0,0.0
7,21,2.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,4.0
2,15,Released,37.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,5.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,36.0,1.0,1.0,9.0,15.0,0.0,0.0
Released,60.0,4.0,37.0,3.0,1.0,28.0,32.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,5.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,36.0,1.0,1.0,9.0,15.0,0.0,0.0
2,Released,60.0,4.0,37.0,3.0,1.0,28.0,32.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,5.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,36.0,1.0,1.0,9.0,15.0,0.0,0.0
2,Released,60.0,4.0,37.0,3.0,1.0,28.0,32.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()